In [ ]:
import geopandas as gpd
import sgis as sg
import dapla as dp
import numpy as np

In [ ]:
VOFSTI = (
    "ssb-prod-vof-data-delt/stedfesting-situasjonsuttak_data/klargjorte-data/parquet"
)
vof_df = dp.read_pandas(f"{VOFSTI}/stedfesting-situasjonsuttak_p2023-01_v1.parquet")
vof_gdf = gpd.GeoDataFrame(
    vof_df,
    geometry=gpd.points_from_xy(
        vof_df["y_koordinat"],
        vof_df["x_koordinat"],
    ),
    crs=25833,
)
vof_gdf

In [ ]:
# Distance example - dont have access. As the crow flies
testdatasti = "ssb-prod-dapla-felles-data-delt/GIS/testdata"
veger = sg.read_geopandas(f"{testdatasti}/veger_oslo.parquet")

boligbygg = sg.read_geopandas(f"{testdatasti}/noen_boligbygg_oslo.parquet")
butikkbygg = sg.read_geopandas(f"{testdatasti}/butikkbygg_oslo.parquet")


# distance as the crow flies
boligbygg_med_butikkavstand = boligbygg.sjoin_nearest(
    butikkbygg, distance_col="meter_til_butikk"
)

In [ ]:
boligbygg_med_butikkavstand

In [ ]:
# distance via roads

veger_klargjort = (
    sg.get_connected_components(veger)
    .query("connected == 1")
    .pipe(sg.make_directed_network_norway, dropnegative=True)
)

regler = sg.NetworkAnalysisRules(directed=True, weight="minutes")

nwa = sg.NetworkAnalysis(network=veger_klargjort, rules=regler)

In [ ]:
# Explore the results in an interactive map

sg.explore(boligbygg, butikkbygg, "meter_til_butikk")

In [ ]:
boligbygg_med_butikkavstand.explore("meter_til_butikk")

In [ ]:
# kommuner

In [ ]:
avstander = nwa.od_cost_matrix(boligbygg, butikkbygg)

boligbygg["minutter_til_butikk"] = avstander.groupby("origin")["minutes"].min()

sg.explore(boligbygg, butikkbygg, "minutter_til_butikk")

In [ ]:
# Thematic maps

# kommuner area

kommuner = sg.read_geopandas(f"{testdatasti}/enkle_kommuner.parquet")

In [ ]:
kommuner

In [ ]:
kommuner_med_boliginfo = kommuner.sjoin(boligbygg, how="left")

boligbygg_med_kommuneinfo = boligbygg.sjoin(kommuner, how="left")

In [ ]:
kommuner["Kvadratmeter"] = kommuner.area

m = sg.ThematicMap(kommuner, column="Kvadratmeter", size=15)
m.title = "Areal i kommunene"
m.plot()

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd


os.chdir("../../src")

import sgis as sg

pd.options.mode.chained_assignment = None
warnings.filterwarnings(action="ignore", category=FutureWarning)

In [ ]:
points = sg.read_parquet_url(
    "https://media.githubusercontent.com/media/statisticsnorway/ssb-sgis/main/tests/testdata/points_oslo.parquet"
)
points = points[["geometry"]]

distances = sg.get_k_nearest_neighbors(points, points, k=100)

mean_distance_above_0 = (
    distances.loc[distances.distance != 0].groupby(level=0)["distance"].mean()
)

points["mean_dist_99_neighbors"] = mean_distance_above_0

print(points)

In [ ]:
m = sg.ThematicMap(sg.buff(points, 100), column="mean_dist_99_neighbors")
m.title = "Distance to 99 nearest neighbors"
m.plot()
m.save("path_to_file.png")

In [ ]:
m = sg.ThematicMap(sg.buff(points, 100), column="mean_dist_99_neighbors")

m.title = "Map with custom (and ugly) colors."
m.title_fontsize = 15
m.title_color = "red"

m.facecolor = "#edf0c9"  # background color

m.change_cmap("PuBuGn", start=20, stop=250)  # start and stop goes from 0 to 256

m.plot()

In [ ]:
# Find k nearest neighbors based on geography alone

# neighbors100  =  sg . get_k_nearest_neighbors (
#     build ,  build ,  k = 100 ,
# )

# neighbors100